In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import word2vec
from konlpy.tag import Okt
from tqdm import tqdm

In [ ]:
!pip install gensim

In [32]:
data = pd.read_csv('data/태그별점추가.csv', encoding='utf-8')
data

,Unnamed: 0,id,title,author,day,genre,synopsis,platform,webtoon_url,thumbnail_url,rate,likes_cnt,tag
0,0,0,참교육,"채용택, 한가람",월,액션,무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다!<부활남> 채...,네이버,https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...,"['9.93', '9.95', '9.96', '9.95', '9.95', '9.95...","20,732","블루스트링, 사이다, 학원물, 다크히어로, 먼치킨, 액션"
1,1,1,뷰티풀 군바리,"설이, 윤성원",월,드라마,'여자도 군대에 간다면?'본격 여자도 군대 가는 만화!,네이버,https://comic.naver.com/webtoon/list?titleId=6...,https://shared-comic.pstatic.net/thumb/webtoon...,"['9.93', '9.83', '9.84', '9.89', '9.91', '9.89...","99,999+","직업드라마, 밀리터리, 드라마"
2,2,2,퀘스트지상주의,박태준 만화회사,월,드라마,"[외모지상주의], [싸움독학], [인생존망]과 세계관을 공유하는 작품!공부, 싸움,...",네이버,https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...,"['9.49', '9.89', '9.81', '9.90', '9.89', '9.68...","1,724","게임판타지, 학원물, 드라마"
3,3,3,장씨세가 호위무사,"김인호, 조형근",월,무협/사극,‘당신이 부른 것이오. 나란 사람을... ’은둔고수 광휘. 호위무사 되다.웹소설 원...,네이버,https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...,"['9.87', '9.95', '9.94', '9.94', '9.87', '9.94...","80,611","사이다, 힘숨찐, 판무, 먼치킨, 무협/사극, 소설원작"
4,4,4,윈드브레이커,조용석,월,스포츠,혼자서 자전거를 즐겨타던 모범생 조자현.원치 않게 자전거 크루의 일에 자꾸 휘말리게...,네이버,https://comic.naver.com/webtoon/list?titleId=6...,https://shared-comic.pstatic.net/thumb/webtoon...,"['8.88', '9.80', '9.89', '9.89', '9.81', '9.91...","99,999+","스포츠성장, 스포츠"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,556,556,밀실 마피아 게임,"WONS, 백지운, PAN",일,스릴러,"평범한 고등학생이었던 강철중, 어느 날 그가 잠에서 깨어나보니 처음 보는 밀실에 갇...",네이버,https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...,"['9.75', '9.68', '9.78', '9.70', '9.63', '9.84...",54,"스릴러, 소설원작"
557,557,557,"아마도, 굿모닝","은나, 솜베, 우지혜",일,로맨스,"""시중은 내가 들죠. 당신이 기뻐할 때까지 최선을 다할 겁니다.""사업 확장을 위한 ...",네이버,https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...,"['9.89', '9.91', '9.92', '9.95', '9.94', '9.95...",52,"로맨스, 소설원작"
558,558,558,천년간 노려왔습니다,수빈,일,로맨스,"용왕에게 토끼의 간을 바치지 못한 죄로, 계속해서 전생의 기억을 갖고 환생을 반복하...",네이버,https://comic.naver.com/webtoon/list?titleId=8...,https://shared-comic.pstatic.net/thumb/webtoon...,"['9.93', '9.89', '9.94', '9.88', '9.92', '9.90...",77,로맨스
559,559,559,그림자 잡기,최날,일,스릴러,"""모든 사건의 시작은 누군가 남겨 놓은 작은 흔적에 걸려 넘어지면서 시작된다. 하지...",네이버,https://comic.naver.com/webtoon/list?titleId=8...,https://shared-comic.pstatic.net/thumb/webtoon...,"['9.67', '8.93', '9.43', '9.55', '9.73', '9.76...",44,스릴러


In [33]:
data['synopsis'] = data['synopsis'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

C:\Users\smhrd\anaconda3\envs\opencv3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [34]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을','로','되다','에게','에서','않다','위해','있다','것']

In [5]:
okt = Okt()

tokenized_data = []
for sentence in tqdm(data['synopsis']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

100%|████████████████████████████████████████████████████████████████████████████████| 561/561 [00:06<00:00, 92.11it/s]


In [6]:
model = word2vec.Word2Vec(sentences=tokenized_data, window = 5, min_count = 5, workers = 4, sg = 0)

In [45]:
model.wv.vectors.shape

(1079, 100)

In [54]:
model.wv.most_similar("강아지")

[('사건', 0.9800432324409485),
 ('반', 0.9799333214759827),
 ('건', 0.979634165763855),
 ('중', 0.9794374704360962),
 ('딸', 0.9791091680526733),
 ('버리다', 0.9790998697280884),
 ('주변', 0.9790725111961365),
 ('여', 0.9789950847625732),
 ('속', 0.9789407849311829),
 ('소녀', 0.9789207577705383)]

In [7]:
import gensim

model = gensim.models.Word2Vec.load('data/ko/ko.bin')

In [8]:
model.wv.save_word2vec_format("ko.bin.gz", binary=False)

In [9]:
from gensim.models import KeyedVectors

In [10]:
model2 = word2vec.Word2Vec(size=200, min_count=1)
model2.build_vocab(tokenized_data)
total_examples = model2.corpus_count
total_examples

561

In [11]:
model2.build_vocab([list(model.wv.vocab.keys())], update=True)
model2.intersect_word2vec_format("ko.bin.gz", binary=False)

In [16]:
model2.most_similar("고양이")

C:\Users\smhrd\anaconda3\envs\opencv3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('토끼', 0.7924244999885559),
 ('원숭이', 0.757001519203186),
 ('강아지', 0.7290453910827637),
 ('돼지', 0.7203201055526733),
 ('개구리', 0.7201221585273743),
 ('족제비', 0.7200244665145874),
 ('악어', 0.7142964005470276),
 ('멧돼지', 0.7096993923187256),
 ('고슴도치', 0.7051361799240112),
 ('거위', 0.6985242366790771)]

In [12]:
model2.train(tokenized_data,total_examples=total_examples,epochs=model2.iter)

C:\Users\smhrd\anaconda3\envs\opencv3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


(118797, 119125)

In [28]:
model2.most_similar("로맨스")

C:\Users\smhrd\anaconda3\envs\opencv3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('코미디', 0.721144437789917),
 ('로맨틱', 0.7021660804748535),
 ('스릴러', 0.6989323496818542),
 ('영화', 0.6428799629211426),
 ('드라마', 0.6342883110046387),
 ('최삡뺩', 0.6271181702613831),
 ('시트콤', 0.619952917098999),
 ('다큐멘터리', 0.6160925626754761),
 ('장편', 0.6126193404197693),
 ('미스터리', 0.6108185648918152)]

In [13]:
for k in model2.wv.vocab:
	result = model.wv.vocab.get(k, "0")
	if result=="0":
		print(k, end=",")

무너지다,지키다,나화,차다,한가람,여자도,간다,존망,잘나다,평범하다,김수현,나타나다,수현,커지다,쉬다,어떻다,해주다,부르다,나다,광휘,웹소설,웰,즐기다,타다,모범생,좋다,현원,꿈꾸다,스트릿,라이딩,위대하다,방옥숙,희다,육아용품,팔다,유부녀,박주연,김예,팔로워,만의,셀럽,엄청나다,누리다,미워하다,올라서다,완벽하다,외롭다,맘씨,착하다,오리가,만나다,겪다,되어다,찌다,파검법,남기다,선근,무투,살다,그렇다,던전,오딘,이라는,얻다,돼다,가지다,받다,비참하다,버리다,르웰린,추악하다,내려지다,막다,열다,얽히다,설키,밝혀지다,누가,살인마,같다,맞추다,살아오다,떠오르다,섬뜩하다,다자,없애다,벗어나다,잔혹하다,우수상,유일하다,주다,당하다,과는,강하다,약하다,빠지다,성공하다,멀다,임파,맛보다,퍼,우승하다,단련하다,다잡다,자임,금슬,에겐,특별하다,그건,걸다,무뚝뚝하다,장난스럽다,스러운,벌어지다,좌충우돌,들어가다,톱스타,셀리나,갑작스럽다,다마,싶다,들어주다,파란만장하다,기다,대장정,펼쳐지다,건강하다,비호감,이준,다물다,채다,한다인,회차,만들다,치다,부술,화려하다,모르다,닥치다,잃다,이를,갈다,닿다,신군,건물붕괴,숨다,드러나다,밀실살인,파헤치다,빛나다,아역배우,연약하다,힘들다,바뀌다,삶다,쓰리다,지언,남다,갑질,시달리다,도윤,부리다,먹다,빨갛다,슈안,차기작,지나다,성숙하다,으른,알다,음란하다,좋아하다,알딸딸하다,귀농,따르다,어쩌다,오가다,이씨,고르다,우당탕,조용하다,그리다,다모,자르다,부족하다,읽다,휩쓸다,초메,빙의,나가다,잖다,얼씬,말고,버티다,돌아가다,드리다,크다,다가오다,빌리다,쓴다,헤어지다,신입사원,바라보다,이상형,공대생,최삡뺩,퍼렁,서민준,에게는,여자친구,사귀다,모태솔로,민준,반하다,싫어하다,숨기다,야하다,높다,새롭다,돌아오다,입부,변하다,상치,생기다,끓다,오르다,프로게이머,이끌다,느끼다,짧다,나오다,사이좋다,삼합회,유석,김남훈,어리다,죽마고우,끈끈,밀다,당기다,성장하다,긴밀하다,만은,놓치다,보내다,상대로,떨치다,아탈란테,로부터,리오넬,비다,죽지,속삭이다,무사하다,술김,아름답

In [23]:
model2.save("ko_new.bin")

In [14]:
model2 = gensim.models.Word2Vec.load('ko_new.bin')
model2.wv.save_word2vec_format("ko_new.bin.gz", binary=False)

In [15]:
model3 = word2vec.Word2Vec(size=200, min_count=1)
model3.build_vocab(tokenized_data)
total_examples = model3.corpus_count
total_examples

561

In [16]:
model3.build_vocab([list(model2.wv.vocab.keys())], update=True)
model3.intersect_word2vec_format("ko_new.bin.gz", binary=False)

In [17]:
model3.train(tokenized_data,total_examples=total_examples,epochs=model3.iter)

C:\Users\smhrd\anaconda3\envs\opencv3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


(119125, 119125)

In [21]:
model3.most_similar("유부녀")

C:\Users\smhrd\anaconda3\envs\opencv3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('코스프레', 0.8642714619636536),
 ('팔다', 0.8613676428794861),
 ('자꾸만', 0.8505290746688843),
 ('이상하다', 0.8457930088043213),
 ('완벽하다', 0.8441935181617737),
 ('최애', 0.8435561656951904),
 ('하필', 0.843131959438324),
 ('쓰다', 0.8420789837837219),
 ('오다', 0.8403727412223816),
 ('같다', 0.8389062881469727)]

In [22]:
for k in model3.wv.vocab:
	result = model2.wv.vocab.get(k, "0")
	if result=="0":
		print(k, end=",")

In [30]:
###############################################################################
###############################################################################
################대충 여기부터 김하랑이 썻다는 뜻 ##############################
###############################################################################
###############################################################################
data2 = pd.read_csv('data/완결웹툰정보ansi.csv', encoding='cp949')
data2

,Unnamed: 0,id,title,author,rate,thumbnail url,synopsis,tag
0,0,0,국선변호사,연재플러스,8.98,https://shared-comic.pstatic.net/thumb/webtoon...,[세상에서 가장 나쁜 방법으로 가장 아름다운 복수를 하는 남자] “난 너 같은 짐...,"스릴러, 완결스릴러"
1,1,1,거래하실래요?,99C / 백도,9.88,https://shared-comic.pstatic.net/thumb/webtoon...,"새로운 사랑은 중고거래로 장만하세요(?!) 남친과 헤어진 후, 추억의 물건을 중고거...","비밀연애, 전남친, 로맨스, 완결무료, 완결로맨스"
2,2,2,지옥급식,둘기마요,9.76,https://shared-comic.pstatic.net/thumb/webtoon...,"힘의 논리로 지배되는 학교, 폭력에 노출된 학생과 방치하는 교사들... 그곳에 양아...","사이다, 학원물, 액션, 완결무료, 완결액션"
3,3,3,팬인데 왜요,요나,9.97,https://shared-comic.pstatic.net/thumb/webtoon...,치열했던 삶이 어느 정도 정리가 되고 이젠 나이가 들어 무기력하게 집안에만 있던 윤...,"2021 지상최대공모전, 힐링, 성장드라마, 드라마, 완결무료, 완결드라마"
4,4,4,모어 라이프,이아영,9.94,https://shared-comic.pstatic.net/thumb/webtoon...,모두가 두 개의 목숨을 가진 세계! 태어날 때부터 목숨이 하나였던 영하는 늘 주변의...,"로맨스, 완결무료, 완결로맨스"
...,...,...,...,...,...,...,...,...
1441,1441,1441,AA,"선정, 지숙",9.68,https://shared-comic.pstatic.net/thumb/webtoon...,대머리 사절을 외치는 여인과 대머리 일것 같은 그녀의 이상형이 벌이는 로맨틱 코미디...,"로맨스, 완결무료, 완결로맨스"
1442,1442,1442,사랑in,전세훈,9.51,https://shared-comic.pstatic.net/thumb/webtoon...,전세훈의 감성터치-사랑in 우리들의 이야기가 수채화 물감처럼 잔잔한 감동으로. 잃어...,"로맨스, 완결로맨스"
1443,1443,1443,까뱅,X-TEAM,8.84,https://shared-comic.pstatic.net/thumb/webtoon...,치킨시티의 새로운 영웅 '까뱅' 오늘도 그의 활약은 계속된다!,"개그, 완결무료, 완결개그"
1444,1444,1444,와탕카,우주인,9.63,https://shared-comic.pstatic.net/thumb/webtoon...,"와탕카='만세','지화자' 좋은 패가 들어오면 외치는 단어 와탕카! 우주인 작가님의...","개그, 완결무료, 완결개그"


In [40]:
data2['sum']= data2['synopsis']+data2['tag']+data2['title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

C:\Users\smhrd\anaconda3\envs\opencv3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [42]:
data2['sum'][0]

'세상에서 가장 나쁜 방법으로 가장 아름다운 복수를 하는 남자  난 너 같은 짐승들을 상대하려고 변호사가 됐지 콩밥도 아까운 녀석에게 증거불충분으로 풀려나게 해서 판결을 내리는 게 내 일이야  이제 시작이다 얼마나 무섭고 고통스러운지 너희들에게 뼈저리게 느끼게 해주지  끝없이 반복되는 거대 악과의 고통스런 싸움 속에서 국선변호사는 과연 정의라는 판결문을 완성할 수 있을 것인가스릴러, 완결스릴러국선변호사'

In [36]:
# 특수 문자 제거해야겠지? 
data2['synopsis'] = data2['synopsis'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

# 불용어 조사, 부사 등등 제거
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에',
             '와','한','하다','을','로','되다','에게','에서','않다','위해','있다','것']

C:\Users\smhrd\anaconda3\envs\opencv3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [37]:
# 정의한 내용 바탕으로 돌아간다아아아
okt = Okt()

tokenized_data2 = []
for sentence in tqdm(data2['synopsis']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data2.append(stopwords_removed_sentence)

100%|█████████████████████████████████████████████████████████████████████████████| 1446/1446 [00:04<00:00, 335.92it/s]


In [38]:
tokenized_data2

[['세상',
  '가장',
  '나쁘다',
  '방법',
  '가장',
  '아름답다',
  '복수',
  '남자',
  '난',
  '너',
  '같다',
  '짐승',
  '상대',
  '변호사',
  '돼다',
  '콩밥',
  '아깝다',
  '녀석',
  '증거불충분',
  '풀다',
  '판결',
  '내리다',
  '게',
  '내',
  '일이',
  '야',
  '이제',
  '시작',
  '이다',
  '얼마나',
  '무섭다',
  '고통스럽다',
  '너희',
  '뼈',
  '저리',
  '게',
  '느끼다',
  '해주다',
  '끝없이',
  '반복',
  '거대',
  '악',
  '과의',
  '고통스럽다',
  '싸움',
  '속',
  '국선',
  '변호사',
  '과연',
  '정의',
  '라는',
  '판결',
  '문',
  '완성',
  '수',
  '인가'],
 ['새롭다',
  '사랑',
  '중고',
  '거래',
  '장만',
  '남친',
  '헤어지다',
  '후',
  '추억',
  '물건',
  '중고',
  '거래',
  '어플',
  '동',
  '네',
  '마켓',
  '파다',
  '한결',
  '하지만',
  '곧',
  '구매',
  '전',
  '남친',
  '이란',
  '걸',
  '알',
  '게',
  '당황',
  '한결',
  '때마침',
  '나타나다',
  '중고',
  '남친',
  '위기',
  '모면',
  '전',
  '남친',
  '강현',
  '새롭다',
  '인연',
  '이안',
  '삼각관계',
  '엔딩',
  '무엇',
  '이다',
  '참새',
  '새',
  '신부',
  '작가',
  '신작'],
 ['힘',
  '논리',
  '지배',
  '학교',
  '폭력',
  '노출',
  '학생',
  '방치',
  '교사',
  '그',
  '곳',
  '양',
  '아치',
  '싫어하다',
  '전학생',
  '오다',
  '안전하다',
  '

In [ ]:
model_hr = word2vec.Word2Vec(size=200, min_count=1)
model_hr.build_vocab(tokenized_data)
total_examples = model3.corpus_count
total_examples
model_hr.build_vocab([list(model3.wv.vocab.keys())], update=True)
model_hr.intersect_word2vec_format("ko_new.bin.gz", binary=False)
model_hr.train(tokenized_data,total_examples=total_examples,epochs=model3.iter)